In [ ]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import inv
from scipy.sparse import csr_matrix
import random

**This is an implementation of the sparsification method introduced by Spielman et al. 2009** <br>
This code calculates a sparsified network approximation for a more dense graph based on effective resistance.
 
Graph Sparsification by Effective Resistance<br>
by Daniel A. Spielman and Nikhil Srivastava

In [ ]:
def create_graph(file_path):
    '''
    Reads file with an edgelist and creates NetworkX graph from it

    Parameters:
        file_path (str): Path to the file containing the edge list.

    Returns:
        G (networkx.Graph): Graph created from the edgelist
    '''
    edges = [] # Empty list to store edges
    with open(file_path, 'r') as file: # Open the file containing the edgelist
        for line in file:
            node1, node2 = line.strip().split('\t') # Split ech line into two nodes
            edges.append((node1, node2)) # Append edge to edgelist
    G = nx.Graph() # Create an empty graph object
    G.add_edges_from(edges) # Add edges to the graph
    return G

In [1]:
def effective_resistance_matrix(G):
    '''
    Computes the effective resistance matrix for a given graph G.

    Parameters:
        G (networkx.Graph()): Graph object for which the effective resistance matrix shall be calculated.

    Returns:
        R (np.array): Effective resistance matrix, where R[i, j] represents the effective resistance between nodes i and j.
    '''
    
    L = nx.laplacian_matrix(G).toarray() # Compute the Laplacian matrix of the graph as a dense array 
    L_pinv = inv(csr_matrix(L).tocsc()) # Compute the pseudoinverse of the Laplacian matrix (sparse matrix is used for efficency)
    
    # Initalize an effective resistance matrix containing only zeros
    n = L.shape[0] # number of nodes in the graph
    R = np.zeros(L.shape)

    # Calculate the effecitve resistance between each node pair
    for i in range(n):
        for j in range(i+1, n):
            # Create indicator vector for nodes i and j
            e_i = np.zeros(n)
            e_i[i] = 1
            e_j = np.zeros(n)
            e_j[j] = 1

            # Calculate the effective resistance according to the formula
            R[i, j] = R[j, i] = (e_i - e_j) @ L_pinv @ (e_i - e_j)
    return R

In [2]:
def sparsify_graph(G, R, epsilon):
    """
    Sparsify an unweighted graph G using effective resistance.
    """
    N = G.number_of_nodes()
    if N < 3:
        raise ValueError('Cannot sparsify a graph with less than 3 nodes')

    # Map nodes to indices
    node_map = {node: idx for idx, node in enumerate(G.nodes())}
    reverse_node_map = {idx: node for node, idx in node_map.items()}

    # Initialize the probability distribution for edge selection
    edges = list(G.edges())
    Pe = np.zeros(len(edges))
    for idx, (u, v) in enumerate(edges):
        u_idx, v_idx = node_map[u], node_map[v]
        Pe[idx] = R[u_idx, v_idx]

    Pe /= np.sum(Pe)  # Normalize to form a probability distribution

    # Number of edges to sample
    C0 = 1 / 30  # This constant can be adjusted
    C = 4 * C0
    q = round(9 * C ** 2 * N * np.log(N) / (epsilon ** 2))

    # Choose random edges based on the probability distribution
    selected_edges_idx = np.random.choice(len(edges), size=q, replace=True, p=Pe)
    selected_edges = [edges[idx] for idx in selected_edges_idx]

    H = nx.Graph()
    H.add_nodes_from(G.nodes())
    for idx in selected_edges_idx:
        u, v = edges[idx]
        H.add_edge(u, v, weight=1/Pe[idx])

    return H

# Sparsifying the graphs

In [1]:
# Specifying the paths were the original, non-sparsified network is found, and where sparsified ones shall
# be saved
non_sparsified_network_path = 'path_to_non_sparsified_network_file'
save_folder = 'folder_to_save_sparsified_networks'

In [ ]:
original_graph = create_graph(non_sparsified_network_path) # Creating a networkx graph object
R = effective_resistance_matrix(L) # Calculating the effective resistance matrix

epsilon = 0.1 # Specify epsilon for how many edges shall be retained
sparsified_graph = sparsify_graph(original_graph, R, epsilon) # Sparsify the graph

In [ ]:
# Save edgelist of sparsified graph if desired
nx.write_edgelist(sparsified_graph, '../00_Data/Sparsified_networks/sparsified_graph.tsv', delimiter='\t', data=True)